# Remarks
This is the implementation of paper : [A Polynomial-Time Algorithm for Solving the Hidden Subset Sum Problem](https://eprint.iacr.org/2020/461.pdf). If you are familiar with Chinese, you can refer to my blog [Orthogonal-Lattice-Attack-HSSP](https://tl2cents.github.io/2023/12/12/Orthogonal-Lattice-Attack/#%E9%9A%90%E8%97%8F%E5%AD%90%E9%9B%86%E5%92%8C%E9%97%AE%E9%A2%98) for details. This is a step by step implementation following the paper.


# Hidden Subset Sum Problem

**Definition $\mathcal{HSSP}$** : Let $M$ be an integer and let $\alpha_1, \alpha_2,\cdots, \alpha_n$ be random integers in $\mathbb{Z}_M$. Let $\rm \bf x_1, x_2, \cdots, x_n\in \mathbb{F_2}^m$ are random binary vectors, and $\rm \bf h = (h1,\cdots, h_m) \in \mathbb{Z}_M^m$ satisfying:

$$
\mathbf{h} = \alpha_1 \mathbf{x_1} + \alpha_2 \mathbf{x_2} + \cdots \alpha_n \mathbf{x_n} \mod M
$$

Given $M$ and $\mathbf{h}$, recover the vector $\boldsymbol{\alpha}=\left(\alpha_1, \ldots, \alpha_n\right)$ and the vectors $\mathbf{x}_i$ 's, up to a permutation of the $\alpha_i$ 's and $\mathbf{x}_i$ 's.


It's worth noting that for each $h_i$, it corresponds to a subset sum problem of size $n$. Therefore, the above vectors provide $m$ subset sum problems on the same number set ${\rm\bf \alpha} = \alpha_1, \alpha_2,\cdots, \alpha_n$, but the number set $\rm\bf \alpha$ is also unknown, hence it is called the Hidden Subset Sum Problem.

In [2]:
from collections.abc import Iterable
from sage.modules.free_module_integer import IntegerLattice
import time

# https://github.com/Neobeo/HackTM2023/blob/main/solve420.sage
# faster LLL reduction to replace `M.LLL()` wiith `flatter(M)`
def flatter(M):
    from subprocess import check_output
    from re import findall
    M = matrix(ZZ,M)
    # compile https://github.com/keeganryan/flatter and put it in $PATH
    z = '[[' + ']\n['.join(' '.join(map(str,row)) for row in M) + ']]'
    ret = check_output(["flatter"], input=z.encode())
    return matrix(M.nrows(), M.ncols(), map(int,findall(b'-?\\d+', ret)))

def gen_hssp(n = 10, m = 20, Mbits = 100):
    M = random_prime(2^Mbits, proof = False, lbound = 2^(Mbits - 1)) 
    # alpha vectors
    a = vector(ZZ, n)
    for i in range(n):
        a[i] = ZZ.random_element(M)
  
    # The matrix X has m rows and must be of rank n
    while True:
        X = random_matrix(GF(2), m, n).change_ring(ZZ)
        if X.rank() == n: break
    
    # Generate an instance of the HSSP: h = X*a % M
    h = X * a % M
    return M, a, X, h

def derive_mod_bits(n):
    iota=0.035
    mod_bits=int(2 * iota * n^2 + n * log(n,2))
    return mod_bits

# Nguyen-Stern algorithm

The Nguyen-Stern algorithm employs the concept of orthogonal lattice to solve the HSSP problem. We consider the (kernel) space orthogonal to the vector $\mathbf{h}$ in $\mathbb{Z}_M$. Suppose $\mathbf{u}$ is orthogonal to $\mathbf{h}$ modulo $M$, which means:

$$
\langle\mathbf{u}, \mathbf{h}\rangle \equiv \alpha_1\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{1}}\right\rangle+\cdots+\alpha_n\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{n}}\right\rangle \equiv 0 \quad(\bmod M)
$$

This implies that $\mathbf{p}_{\mathbf{u}}=\left(\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{1}}\right\rangle, \ldots,\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{n}}\right\rangle\right)$ and vector $\boldsymbol{\alpha}=\left(\alpha_1, \ldots, \alpha_n\right)$ are orthogonal modulo $M$. 

We look for a sufficiently short vector $\mathbf{u}$. Since $\rm \bf x_1, x_2, \cdots, x_n\in \mathbb{F_2}^m$, this would result in a short $\mathbf{p}_{\mathbf{u}}$ if $\mathbf{u}$ is short. We know that in the kernel space orthogonal to $\boldsymbol{\alpha}=$ $\left(\alpha_1, \ldots, \alpha_n\right)$ modulo $M$, there exists a shortest vector, and let the 2-norm of the shortest vector in this kernel space be $\gamma$. If the vector $\mathbf{u}$ is very short, with a 2-norm far less than $\gamma$, then $\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{i}}\right\rangle$ is also far less than $\gamma$. That is, the 2-norm of $\mathbf{p}_{\mathbf{u}}=\left(\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{1}}\right\rangle, \ldots,\left\langle\mathbf{u}, \mathbf{x}_{\mathbf{n}}\right\rangle\right)$ is less than $\gamma$ and orthogonal to $\boldsymbol{\alpha}$. Then it can only satisfy $\mathbf{p}_{\mathbf{u}}= \mathbf{0}$. **Therefore, the $\mathbf{u}$ we obtain will be orthogonal to all vectors $\rm \bf x_1, x_2, \cdots, x_n$ in the integer ring**.

The following lattice can obatin an LLL-reduced basis of $\mathcal{Ker}(\mathbf{h})$ modulo $M$ ($\beta$ sufficiently large):

$$
\mathcal{L(\mathbf{h}^\perp)}_M = 
\left(\begin{array}{cccc}
\beta h_1 & 1 & & \\
\beta h_2 &  & 1 & \\
\vdots & & & \ddots  \\
\beta h_n & & & & 1 \\
\beta M
\end{array}\right)_{(n+1) \times(n+1)}
$$

If we want to compute it in integer ring, just remove the modulo row vector:

$$
\mathcal{L(\mathbf{h}^\perp)}_{Z} = 
\left(\begin{array}{cccc}
\beta h_1 & 1 & & \\
\beta h_2 &  & 1 & \\
\vdots & & & \ddots  \\
\beta h_n & & & & 1 \\
\end{array}\right)_{n \times(n+1)}
$$

A generic implementation :

In [6]:
# compute kernel space and obtain an LLL-reduced basis
def kernel_LLL(vec_mat, mod=None, verbose=True, w=None):
    """
    Input :
    vec_mat : m * n matrix, the m vectors are : v1,..., vm in Z^n
    mod     : if mod is not None, we find kernel in Zmod(mod) else in ZZ
    Output :
    B : matrix, an LLL-reduced basis b1,b2,...,bk such that bi * vj for all i in [1,k], j in [1,m]   
    """
    m, n = vec_mat.dimensions()
    if mod is None:
        # if n < 2*m : return vec_mat.right_kernel().matrix()
        if w is None : w=2^(n//2)* vec_mat.height()
        M = block_matrix(ZZ, [w * vec_mat.T, 1], ncols = 2).dense_matrix()
    else:
        if w is None : w = 2^(ZZ(mod).nbits())
        M = block_matrix(ZZ, [
            [w * vec_mat.T, 1],
            [w * mod, 0]
        ]).dense_matrix()
    if verbose: print(f"    [*] start to LLL reduction with dim {M.dimensions()}")
    # L = M.LLL()
    t0 = time.time()
    L = flatter(M)
    t1 = time.time()
    if verbose: print(f"    [+] LLL reduction done in {t1 -t0}")
    # filter orthogonal vectors
    basis = []
    for row in L:
        if row[:m] == 0:
            # orthogonal vector
            basis.append(row[m:m+n])
    if verbose: print(f"    [+] find {len(basis)} orthogonal vectors for {m} {n}-dim vectors")
    return matrix(ZZ, basis)

def test_kernel_LLL():
    print("[+] kernel_LLL testing...")    
    p = random_prime(2**128)
    M = random_matrix(GF(p), 10, 20)
    M = M.change_ring(ZZ)
    KerZ = kernel_LLL(M, verbose=True)
    assert M * KerZ.T == 0, "not orthogonal"
    KerP = kernel_LLL(M, mod=p, verbose=True)
    assert M * KerP.T % p == 0, "not orthogonal"
    print("[+] kernel_LLL passed")
    
test_kernel_LLL()

[+] kernel_LLL testing...
    [*] start to LLL reduction with dim (20, 30)
    [+] LLL reduction done in 0.0739138126373291
    [+] find 10 orthogonal vectors for 10 20-dim vectors
    [*] start to LLL reduction with dim (30, 30)
    [+] LLL reduction done in 0.14871883392333984
    [+] find 20 orthogonal vectors for 10 20-dim vectors
[+] kernel_LLL passed


## Step1. OL Attack
Given $\mathbf{h}, M$ from $\mathcal{HSSP}$, we can recover the original basis of $\mathcal{X}$. We compute two kernel LLL. the first one in $\mathbf{Z}_M$, the second in $\mathbf{Z}$.

Let :

$$
\mathcal{L}_0:=\Lambda_M^{\perp}(\mathbf{h})=\left\{\mathbf{u} \in \mathbb{Z}^m \mid\langle\mathbf{u}, \mathbf{h}\rangle \equiv 0 \quad(\bmod M)\right\} \\
\mathcal{L}_{\mathbf{x}} = \mathcal{L}(\rm\bf x_1,x_2,\cdots, x_n)
$$

The OL (Orthogonal Lattice) attack of Nguyen-Stern algorithm is as follows:

<center><img src="./img/image.png" alt="ol attak" style="zoom: 65%;" /></center>


In [7]:
def ol_attack(h, m, n , M, verbose=False):
    """
    HSSP : h = X * a  % M
    Input :
    h : hssp result m-dim vector
    m,n : X.dimensions()
    M : the mod
    Output:
    the basis b1, ..., bn generating x1,...,xn (column vectors of X)
    """
    H = matrix(ZZ,h)
    # we only need m - n generating basis
    basis = kernel_LLL(H, mod = M, verbose=verbose)[:m - n]
    # check
    assert basis * H.T % M == 0, "not kernel, do check"
    # the basis is orthogonal to x_i in ZZ by assumption
    # try to recover the basis of xi
    xi_basis = kernel_LLL(basis, verbose=verbose)
    return xi_basis

def test_ol_attack():
    print("[+] test first step: OL attack")    
    n = 64
    m = 128
    Mbits = derive_mod_bits(n)
    M, a, X, h = gen_hssp(n, m, Mbits)
    C = ol_attack(h, m, n , M, verbose=True)
    X_lattice = IntegerLattice(X.T)
    for ci in C:
        assert ci in X_lattice
    print("[+] basis recovered")
    
test_ol_attack()

[+] test first step: OL attack
    [*] start to LLL reduction with dim (129, 129)
    [+] LLL reduction done in 1.680335521697998
    [+] find 128 orthogonal vectors for 1 128-dim vectors
    [*] start to LLL reduction with dim (128, 192)
    [+] LLL reduction done in 1.4927475452423096
    [+] find 64 orthogonal vectors for 64 128-dim vectors
[+] basis recovered


## Step 2. Recovering binary basis

By doing BKZ-reduction, we can finally recover short basis of the recovered $\mathcal{L}_{\mathbf{x}}$. We hope that we can find the shortest vectors and they are exactly the binary vectors $\mathbf{x_1,\cdots, x_n}$. However, recovering the vectors $\rm\bf x_1,x_2,\cdots,x_n$ with the Nguyen-Stern algorithm isn't straightforward because $\mathbf{x_i}$ isn't necessarily the shortest vector. It could be a linear combination of them, for example, $\rm\bf {x}_i - {x_j}$ might be shorter than $x_i$. The elements of the vector obtained through reduction are likely to fall into $\{0,1,-1\}$, while the elements of our target vector $\rm\bf x_i$ will only fall into $\{0,1\}$.

Therefore, the Nguyen-Stern algorithm proposes transforming the lattice $\mathcal{L}_{\mathrm{x}}$ into $\mathcal{L}_{\mathrm{x}}^{\prime}=2 \mathcal{L}_{\mathrm{x}}+\mathbf{e} \mathbb{Z}$, where $\mathbf{e}=(1, \ldots, 1)$. This means for a vector $\mathbf{v} \in \mathcal{L}_{\mathrm{x}}$ whose elements fall into $\{0,1,-1\}$, its corresponding short vector in the new lattice is $2 \mathbf{v} \in \mathcal{L}_{\mathbf{x}}^{\prime}$ (adding or subtracting several $\mathbf{e}$ cannot reduce the modulus), and its elements will fall into $\{0,-2,2\}$. For a vector $\mathbf{v} \in \mathcal{L}_{\mathrm{x}}$ whose elements only fall into $\{0,1\}$, its corresponding short vector in the new lattice is $2 \mathbf{x}-\mathbf{e} \in \mathcal{L}_{\mathbf{x}}^{\prime}$, and its elements will fall into $\{-1,1\}$, which is likely to be shorter than the previous vector.

Therefore, in the new lattice $\mathcal{L}_{\mathrm{x}}^{\prime}$, we can recover the original vectors $\rm\bf x_1,x_2,\cdots,x_n$, rather than their linear combinations.

In this paper, the authors propose a new greedy method to recover the target vector $\mathbf{x_i}$ since there exists original $\mathbf{x_i}$ in the BKZ-reduced basis with high probability. They search these vectors and do adding and subtracting with other vectors to recover other vectors until we can not find more.

I implement those two methods as follows:

In [12]:
def check_matrix(M, white_list = [1,0,-1]):
    # check wheter all values in M fall into white_list
    for row in M:
        for num in row:
            if num not in white_list:
                return False
    return True

def all_ones(v):
    if len([vj for vj in v if vj in [0,1]])==len(v):
        return v
    if len([vj for vj in v if vj in [0,-1]])==len(v):
        return -v
    return None
    
def recover_binary_basis(basis):
    lv = [all_ones(vi) for vi in basis if all_ones(vi)]
    n = basis.nrows()
    for v in lv:
        for i in range(n):
            nv = all_ones(basis[i] - v)
            if nv and nv not in lv:
                lv.append(nv)
            nv = all_ones(basis[i] + v)
            if nv and nv not in lv:
                lv.append(nv)
    return matrix(lv)

def find_original_basis(basis, new_basis):
    n, m = basis.dimensions()
    origin_lattice = IntegerLattice(basis)
    origin_basis = []
    for row in new_basis:
        if sum(row) == m:
            continue
        # seems like we cannot determine wether 1,-1 represents 1 or 0 in this lattcie
        # therefore we do some checking in the original lattice
        v = vector(ZZ, [1 if num == 1 else 0 for num in row])
        if v in origin_lattice:
            origin_basis.append(v)
        else:
            v = vector(ZZ, [0 if num == 1 else 1 for num in row])
            assert v in origin_lattice, "oops, something wrong"
            origin_basis.append(v)
    return matrix(ZZ, origin_basis)


def recover_binary_basis_by_lattice(basis, blocksize = None):
    new_lattice = 2 * basis
    n, m = basis.dimensions()
    new_lattice = new_lattice.insert_row(0, [1] * m)
    if blocksize is None: 
        # new_basis = new_lattice.LLL()
        new_basis = flatter(new_lattice)
    else:
        new_basis = new_lattice.BKZ(block_size = blocksize)
    
    if not check_matrix(new_basis, [1,-1]):
        print("[+] fails to recover basis")
        return None
        
    origin_lattice = IntegerLattice(basis)
    origin_basis = []
    for row in new_basis:
        if sum(row) == m:
            continue
        # seems like we cannot determine wether 1,-1 represents 1 or 0 in this lattcie
        # therefore we do some checking in the original lattice
        v = vector(ZZ, [1 if num == 1 else 0 for num in row])
        if v in origin_lattice:
            origin_basis.append(v)
        else:
            v = vector(ZZ, [0 if num == 1 else 1 for num in row])
            assert v in origin_lattice, "oops, something wrong"
            origin_basis.append(v)
    return matrix(ZZ, origin_basis)

# Nguyen-Stern attack using greedy method mentioned in appendix D of https://eprint.iacr.org/2020/461.pdf
def ns_attack_greedy(h, m, n, M, bkz = range(2,12,2), verbose=True):
    t0 = time.time()
    if verbose: print(f"[*] start to ns attack with greedy method")
    xi_basis = ol_attack(h, m, n, M)
    assert isinstance(bkz, Iterable), "give a list or iterable object as block_size para"
    L = xi_basis
    if verbose: print(f"    [+] basis dimensions : {L.dimensions()}")
    assert L.dimensions() == (n,m) , "basis generating xi's is not fully recovered"
    for bs in bkz:
        if verbose: print(f"    [*] start to BKZ reduction with block_size {bs}")
        L = L.BKZ(block_size = bs)
        if verbose: print(f"    [+] BKZ reduction with block_size {bs} done")
        if check_matrix(L,[-1,1,0]):
            if verbose: print("    [+] find valid basis")
            break
            
    XT = recover_binary_basis(L)
    if verbose: print(f"    [+] Number of recovered xi vectors {XT.nrows()}")
    if XT.nrows() < n:
        print(f"    [+] not enough xi vectors recovered, {XT.nrows()} out of {n}")
        print(f"    [*] trying new lattice recovery...")
        XT = recover_binary_basis_by_lattice(L)
        if XT.nrows() < n:
            print(f"[+] failed.")
            return False, L
    X =  XT.T
    # h = X * a
    a = matrix(Zmod(M) ,X[:n]).inverse() * vector(Zmod(M),h[:n])
    t1 = time.time()
    if verbose : print(f"    [+] total time cost in {t1 - t0}")
    return True, a

# Nguyen-Stern attack using 2*Lx + E lattice method
def ns_attack_2Lx(h, m, n, M, bkz = range(2,12,2), verbose=True):
    t0 = time.time()
    if verbose: print(f"[*] start to ns attack with 2*Lx + E method")
    xi_basis = ol_attack(h, m, n, M)
    assert isinstance(bkz, Iterable), "give a list or iterable object as block_size para"
    # we use the new lattice : 2 * basis + [1, ..., 1], the final vectors all fall into [-1, 1]
    L = 2 * xi_basis
    L = L.insert_row(0, [1] * m)
    if verbose: print(f"    [+] basis dimensions : {L.dimensions()}")
    assert L.dimensions() == (n + 1, m) , "basis generating xi's is not fully recovered"
    for bs in bkz:
        if verbose: print(f"    [*] start to BKZ reduction with block_size {bs}")
        L = L.BKZ(block_size = bs)
        if verbose: print(f"    [+] BKZ reduction with block_size {bs} done")
        if check_matrix(L,[-1,1]):
            if verbose: print("    [+] find valid basis")
            break
            
    XT = find_original_basis(xi_basis, L)
    if verbose: print(f"    [+] Number of recovered xi vectors {XT.nrows()}")
    if XT.nrows() < n:
        return False, L
    X =  XT.T
    # h = X * a
    a = matrix(Zmod(M) ,X[:n]).inverse() * vector(Zmod(M),h[:n])
    t1 = time.time()
    if verbose : print(f"    [+] total time cost in {t1 - t0}")
    return True, a

In [16]:
n = 100
m = 200
Mbits = derive_mod_bits(n)
M, a, X, h = gen_hssp(n, m, Mbits)

In [17]:
find, recovered_a = ns_attack_greedy(h, m, n, M, range(2,32,4))
if find:
    print(f"[+] check {sorted(a) == sorted(recovered_a)}")    

[*] start to ns attack with greedy method
    [+] basis dimensions : (100, 200)
    [*] start to BKZ reduction with block_size 2
    [+] BKZ reduction with block_size 2 done
    [*] start to BKZ reduction with block_size 6
    [+] BKZ reduction with block_size 6 done
    [*] start to BKZ reduction with block_size 10
    [+] BKZ reduction with block_size 10 done
    [+] find valid basis
    [+] Number of recovered xi vectors 100
    [+] total time cost in 20.982471466064453
[+] check True


In [18]:
find, recovered_a = ns_attack_2Lx(h, m, n, M, range(2,32,4))
if find:
    print(f"[+] check {sorted(a) == sorted(recovered_a)}")    

[*] start to ns attack with 2*Lx + E method
    [+] basis dimensions : (101, 200)
    [*] start to BKZ reduction with block_size 2
    [+] BKZ reduction with block_size 2 done
    [*] start to BKZ reduction with block_size 6
    [+] BKZ reduction with block_size 6 done
    [+] find valid basis
    [+] Number of recovered xi vectors 100
    [+] total time cost in 18.139620542526245
[+] check True


# Multivariate Attack

Jean-Sébastien Coron and Agnese Gini modified the final BKZ reduction part of the Nguyen-Stern algorithm to recover the vectors $\rm\bf x_1,x_2,\cdots,x_n$, using the idea of linearization and multivariate equations to recover the target vectors $\rm\bf x_1,x_2,\cdots,x_n$. In the Nguyen-Stern algorithm, we only need about $m = 2n$ hidden subset and samples to solve, but the optimized algorithm needs at least $m = n^2/2$ hidden subset and samples to solve the multivariate equation.

## New Orthogonal Lattice Computation

Therefore, a practical problem is that we can't directly use the algorithm to get $\bar{\mathcal{L}}_{\mathrm{x}}$ by LLL. When $n = 200$, $m =20000$, the dimension is too large and the performance of LLL is extremely poor. Even when $M$ is not a prime number and is difficult to decompose, the solution of the kernel space is quite difficult. So Jean-Sébastien Coron and Agnese Gini proposed a clever way to compute $\bar{\mathcal{L}}_{\mathrm{x}}$ in such a very high-dimensional scenario. They only perform LLL-reduce on the lattice of size $2n$ each time, obtain the projection of the complete basis on the corresponding dimension, then select the first $n$ dimensions (fixed), and perform simple transformation calculations on other dimensions. In this way, the results of multiple calculations can be combined in a relatively simple way to obtain the final $\bar{\mathcal{L}}_{\mathrm{x}}$. A conceptual calculation flowchart is as follows.

<center><img src="./img/image-1.png" alt="Alt text" style="zoom: 40%;" /></center>

Assume $m = (d+1)n$, $\mathbf{h}=\left[\mathbf{h}_0, \ldots, \mathbf{h}_d\right]$, where $\mathbf{h_i} \in \mathbb{Z}^n$. Each time, use algorithm 1 to calculate the projection of $\bar{\mathcal{L}}_{\mathrm{x}}$ on the corresponding dimension for this sub-vector $\left(\mathbf{h}_0, \mathbf{h}_i\right) \in \mathbb{Z}^{2 n}$, and get $\mathbf{C}_0^{(i)} \| \mathbf{C}_i \in \mathbb{Z}^{n \times 2 n}$. But we ensure that the projection of the first $n$ dimensions is always fixed as $\mathbf{C}_0 = \mathbf{C}_0^{(1)}$, and then we can combine the projections of the basis of $\bar{\mathcal{L}}_{\mathrm{x}}$ on various dimensions. Represent algorithm 1 as a function, denoted as $\textsf{OrthoLat}$. Then the orthogonal lattice attack algorithm for the case of $m \gg n$ is as follows.

<center><img src="./img/image-2.png" alt="Alt text" style="zoom:40%;" /></center>

In [23]:
from tqdm import tqdm
import time 

# implement the new multivariate attack with m//n LLL reductions
# algorithm 2 introduced in section 4.1 of https://eprint.iacr.org/2020/461.pdf 
# the paper implementations uses optimization mentioned in appendix F
def multivariate_ol_attack_step1(h, m, n, M, verbose=False):
    # return basis that can generate X, i.e. return basis_matrix of L(X)
    # m = d * m
    d = m // n
    h = list(h)
    hs = [h[i*n : (i+1)*n] for i in range(d)]
    Cs = []
    if verbose:
        iter_item = tqdm(range(d - 1))
    else:
        iter_item  = range(d - 1)
    for i in iter_item:
        yi = vector(ZZ, hs[0] + hs[i+1])
        C = ol_attack(yi, 2*n, n, M)[:n]
        if i == 0:
            C0 = C[:, :n]
            C1 = C[:, n:]
            C0_inv = C0
            Cs += [C0, C1]
        else:
            C0_i = C[:, :n]
            Ci = C[:, n:]
            Ci = C0 * C0_i^(-1) * Ci
            Cs.append(Ci)
    return block_matrix(ZZ, Cs, ncols = len(Cs))

## Optimized LLL-reduced Orthogonal Vectors Computation

In above process, we needs about $n$ LLL reductions. The authors then used Babai's rounding method to reduce the times of LLL reduction to 1.

First, we can use the following lattice to recover first n vectors orthogonal to $h$.

$$
\mathcal{L}_0:\left(\begin{array}{ccccccc}
M & & & & \\
h_2 h_1^{-1} & 1 & & &\\
\vdots & & \ddots \\
h_{2 n} h_1^{-1} & & & 1 \\
\end{array}\right)
\\
\implies \mathcal{L}_0.LLL(): \mathbf{a_1, \cdots, a_{2n}}
$$

Then we can use the Babai's rounding method to compute the other short vectors orthogonal to $\mathbf{h}$. The process is as follows:


<center><img src="./img/multi1.png" alt="Alt text" style="zoom:70%;" /></center>

The detailed algorithm :

<center><img src="./img/multi2.png" alt="Alt text" style="zoom:70%;" /></center>


In [25]:
def sub_kernel_LLL(h, M, verbose=False):
    # h = [h1 * h0_inv, h2 * h0_inv, ... , h2n* h0_inv]
    # return kernel space orthogonal to [h0, h1, ..., h2n] mod M
    t0 = time.time()
    n = len(h)
    L = identity_matrix(ZZ, n + 1)
    L[0,0] = M
    for i in range(n):
        L[i+1, 0] = h[i]
    L = flatter(L)
    # L = L.LLL()
    t1 = time.time()
    if verbose: print(f"    [+] sub kernel LLL with dim ({n+1, n+1}) costs {t1 - t0} seconds")
    return L

# implement kernel space computation with only one LLL reduction when m >> n
# algorithm 5 introduced in section 5.2 of https://eprint.iacr.org/2020/461.pdf 
def optimized_lll_reduced_orthogonal_vectors(h, m, n, M, verbose=False):
    t0 = time.time()
    if verbose: print(f"    [+] start to compute basis orthogonal to h")
    h0 = h[0]
    H1 = vector(ZZ, h[1:]) * ZZ(inverse_mod(-h0, M)) % M
    M_2n_basis = sub_kernel_LLL(H1.list()[:2 * n - 1], M, verbose)
    # THIS COSTS A LOT OF TIME !!!
    RF = RealField(2 * ZZ(M).nbits())
    M_2n_INV = matrix(RF, M_2n_basis).inverse()
    ortho_vecs = [list(vec) + [0] * (m - 2 * n) for vec in M_2n_basis[:n]]
    for i in range(m - 2 * n):
        idx = 2*n + i
        t = vector(ZZ, [H1[idx - 1]] + (2 * n -1) * [0])
        # Babai rounding process
        u = list(t[0] * M_2n_INV[0])
        u_vec = vector(ZZ, [round(num) for num in u])
        v_vec = vector(ZZ, u_vec * M_2n_basis)
        ai_2n = (t  - v_vec)
        ai_prime = list(ai_2n) + [1 if j==i else 0 for j in range(m - 2 * n)]
        ortho_vecs.append(ai_prime)
    t1 = time.time()
    if verbose: print(f"    [+] basis orthogonal to h computed in {t1 - t0} seconds")
    return matrix(ZZ, ortho_vecs)

# implement the new multivariate attack step 1 with only one LLL reduction
def optimized_multivariate_ol_attack_step1(h, m, n, M, verbose=False):
    # return basis that can generate X^T with size n * m, i.e. return basis_matrix of L(X^T)
    # return a matrix M with size n * m such that L(M) =  L(X^T)
    orthogonal_h_basis = optimized_lll_reduced_orthogonal_vectors(h, m, n, M, verbose=verbose)
    # orthogonal_h_basis is almost in Hermite Normal Form (after the first 2n components)
    # therefore we will use a straightforward and fast way to compute its kernel
    U = orthogonal_h_basis[:n, :2*n]
    V = orthogonal_h_basis[n:, :2*n]
    P = kernel_LLL(U, verbose=verbose).T
    C = block_matrix(ZZ, [P, - V * P], nrows = 2)
    assert orthogonal_h_basis * C == 0, "not orthogonal"
    return C.T

# implement the new multivariate attack step 1 with only one LLL reduction
def optimized_multivariate_ol_attack_step1_GF3(h, m, n, M, verbose=False):
    # return basis that can generate X^T with size n * m, i.e. return basis_matrix of L(X^T)
    # return a matrix M with size n * m such that L(M) =  L(X^T)
    orthogonal_h_basis = optimized_lll_reduced_orthogonal_vectors(h, m, n, M, verbose=verbose)
    # orthogonal_h_basis is almost in Hermite Normal Form (after the first 2n components)
    # therefore we will use a straightforward and fast way to compute its kernel
    U = orthogonal_h_basis[:n, :2*n]
    V = orthogonal_h_basis[n:, :2*n]
    P = kernel_LLL(U, verbose=verbose).T
    P3 = P.change_ring(GF(3))
    V3 = V.change_ring(GF(3))
    C = block_matrix(GF(3), [P3, - V3 * P3], nrows = 2)
    return C.T

def test_multi_step1():
    n = 32

    if n % 2==1:
        m = n * (n + 3) // 2 # n is odd
    else:
        m = n * (n + 4) // 2 # n is even

    Mbits = derive_mod_bits(n)
    M, a, X, h  = gen_hssp(n, m, Mbits)

    C = multivariate_ol_attack_step1(h, m, n, M)
    XT_lattice = IntegerLattice(X.T)
    for row in C:
        assert (row in XT_lattice)
    print("[+] ok with plain multivariate_ol_attack_step1")

    C1 = optimized_multivariate_ol_attack_step1(h, m, n, M)

    XT_lattice = IntegerLattice(X.T)
    for row in C1:
        assert (row in XT_lattice)
    print("[+] ok with optimized_multivariate_ol_attack_step1")
    
test_multi_step1()

[+] ok with plain multivariate_ol_attack_step1
[+] ok with optimized_multivariate_ol_attack_step1


## Solve Multivariate Equations

**Problem 2**. Given $\mathbf{C} \in \mathbb{Z}^{n \times m}$ of rank $n$, suppose there exist exactly $n$ vectors $\mathbf{w}_i \in \mathbb{Z}^n$ such that $\mathbf{w}_i \cdot \mathbf{C}=\mathbf{x}_i \in\{0,1\}^m$ for $i=1, \ldots, n$, and assume that the vectors $\mathbf{w}_i$ are linearly independent. Find $\mathbf{w}_1, \ldots, \mathbf{w}_n$.
We denote by $\tilde{\mathbf{c}}_1, \ldots, \tilde{\mathbf{c}}_m$ the column vectors of $\mathbf{C}$, which gives:
$$
\left[\begin{array}{c}
\mathbf{w}_1 \\
\vdots \\
\mathbf{w}_n
\end{array}\right]\left[\begin{array}{lll} 
& & \\
\tilde{\mathbf{c}}_1 & \cdots & \tilde{\mathbf{c}}_m \\
& & \\
\end{array}\right]=\left[\begin{array}{c}
\mathbf{x}_1 \\
\vdots \\
\mathbf{x}_n
\end{array}\right]
$$

**Multivariate approach**. The crucial observation is that since all components of the vectors $\mathbf{x}_i$ are binary, they must all satisfy the quadratic equation $y^2-y=0$. Therefore for each $i=1, \ldots, n$ we have:
$$
\begin{aligned}
\mathbf{w}_i \cdot \mathbf{C} \in\{0,1\}^m & \Longleftrightarrow \forall j \in[1, m],\left(\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j\right)^2-\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j=0 \\
& \Longleftrightarrow \forall j \in[1, m],\left(\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j\right)\left(\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j\right)^{\top}-\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j=0 \\
& \Longleftrightarrow \forall j \in[1, m], \mathbf{w}_i \cdot\left(\tilde{\mathbf{c}}_j \cdot \tilde{\mathbf{c}}_j^{\top}\right) \cdot \mathbf{w}_i^{\top}-\mathbf{w}_i \cdot \tilde{\mathbf{c}}_j=0
\end{aligned}
$$

Given the known column vectors $\tilde{\mathbf{c}}_1, \ldots, \tilde{\mathbf{c}}_m$, the vectors $\mathbf{w}_1, \ldots, \mathbf{w}_n$ and $\mathbf{0}$ are therefore solutions of the quadratic polynomial multivariate system
$$
\left\{\begin{array}{c}
\mathbf{w} \cdot \tilde{\mathbf{c}}_1 \tilde{\mathbf{c}}_1^{\top} \cdot \mathbf{w}^{\top}-\mathbf{w} \cdot \tilde{\mathbf{c}}_1=0 \\
\vdots \\
\mathbf{w} \cdot \tilde{\mathbf{c}}_m \tilde{\mathbf{c}}_m^{\top} \cdot \mathbf{w}^{\top}-\mathbf{w} \cdot \tilde{\mathbf{c}}_m=0
\end{array}\right.
$$

**Linearization.** Since $\left(\tilde{\mathbf{c}}_j\right)_i=\mathbf{C}_{i j}$, for all $1 \leq j \leq m$, we can write:
$$
\mathbf{y} \cdot \tilde{\mathbf{c}}_j \tilde{\mathbf{c}}_j^{\top} \cdot \mathbf{y}^{\boldsymbol{\top}}=\sum_{i=1}^n \sum_{k=1}^n y_i y_k \mathbf{C}_{i j} \mathbf{C}_{k j}=\sum_{i=1}^n \sum_{k=i}^n y_i y_k\left(2-\delta_{i, k}\right) \mathbf{C}_{i j} \mathbf{C}_{k j}
$$
with $\delta_{i, k}=1$ if $i=k$ and 0 otherwise. In the above equation the coefficient of the degree 2 monomial $y_i y_k$ for $1 \leq i \leq k \leq n$ is $\left(2-\delta_{i, k}\right) \mathbf{C}_{i j} \mathbf{C}_{k j}$. Thus, we consider the corresponding vectors of coefficients for $1 \leq j \leq m$ :
$$
\mathbf{r}_j=\left(\left(2-\delta_{i, k}\right) \mathbf{C}_{i j} \mathbf{C}_{k j}\right)_{1 \leq i \leq k \leq n} \in \mathbb{Z}^{\frac{n^2+n}{2}} .
$$

We set $\mathbf{R} \in \mathbb{Z}^{\frac{n^2+n}{2} \times m}$ to be the matrix whose columns are the $\mathbf{r}_j$ 's and
$$
\mathbf{E}=\left[\frac{\mathbf{R}}{-\mathbf{C}}\right] \in \mathbb{Z}^{\frac{n^2+3 n}{2} \times m} ;
$$
we obtain that the original problem is equivalent to
$$
\left\{\begin{array}{l}
{[\mathbf{z} \mid \mathbf{y}] \cdot \mathbf{E}=0} \\
\mathbf{z}=\left(y_i y_k\right)_{1 \leq i \leq k \leq n} \in \mathbb{Z}^{\frac{n^2+n}{2}}
\end{array}\right.
$$

Then compute its kernel $\mathbf{K}$ in systematic form and transform the problem to compute the eigenspace space of submatrix of $\mathbf{K}$, we can recover the orignal solutions of $\mathbf{x_i}$. One can refer to the paper for more details.

In [28]:
# algorithm 3 introduced in section 4.2 of https://eprint.iacr.org/2020/461.pdf 
def _multivariate_attack(C, m, n, M, verbose=False):
    """
    input
    `C`: a given basis with size n * m of x1,...,xn
    return : x1,x2,...,xn in {0,1}^m  , such that L(x1,...,xn) = L(C) i.e. wi C = xi for i = 1,..,n
    """
    # turn matrix into GF(3)
    t1 = time.time()
    C = matrix(GF(3), C)
    mod = M
    # linearization matrix R: rj = ((2-delta_{i,k}) Cij Ckj) 1<=i<=k<=n
    R = matrix(GF(3), m, ZZ((n**2 + n)/2))
    look_up_dict = {}
    for j in range(m):
        idx = 0
        for i in range(0, n):
            for k in range(i,n):
                delta_ik = int(i==k)
                R[j, idx] = (2 - delta_ik) * C[i,j] * C[k, j]
                look_up_dict[tuple((i,k))] = idx 
                idx += 1
    E = block_matrix(GF(3), [R.T, - C], nrows = 2)
    ker = E.left_kernel().matrix()
    t2 = time.time()
    if verbose: print(f"    [+] {ker.dimensions() = } , R matrix construction and kernel computation cost {t2-t1} seconds")
    assert ker.dimensions() == (n, ZZ((n**2 + 3 * n)/2)), "weird dimensions"
    # turn kernel matrix into systematic form
    K1 = ker[:, :-n]
    K2 = ker[:, -n:]
    M = (K2^-1) * K1
    # duplicating columns of M
    M_prime =  matrix(GF(3),n, n**2)
    for i in range(n):
        for k in range(n):
            if i <= k:
                idx = look_up_dict[tuple((i,k))] 
            else:
                idx = look_up_dict[tuple((k,i))]
            M_prime[:,i*n + k] = M[:,idx]
    t3 = time.time()
    if verbose: print(f"    [+] systematic matrix and M_prime matrix constructed in {t3 -t2} seconds.")
    # We will compute the list of eigenvectors
    # We start with the full space.
    # We loop over the coordinates. This will split the eigenspaces.
    L1 = [Matrix(GF(3),identity_matrix(n))]    
    for j in range(n):               # We loop over the coordinates of the wi vectors.
        M = M_prime[:,j*n:(j+1)*n]   # We select the submatrix corresponding to coordinate j
        L2 = []                      # We initialize the next list
        for v in L1:
            if v.nrows()==1:         # We are done with this eigenvector 
                L2.append(v)
            else:                    # eigenspace of dimension >1
                A = v.solve_left(v * M)
                # v*M=A*v. When we apply M on the right, this is equivalent to applying the matrix A.
                # The eigenvalues of matrix A correspond to the jth coordinates of the wi vectors in that eigenspace
                for e, v2 in A.eigenspaces_left():    # We split the eigenspace according to the eigenvalues of A.
                    v2_mat = v2.matrix()
                    L2.append(v2_mat * v)                   # The new eigenspaces 
        L1 = L2
    t4 = time.time()
    if verbose: print(f"    [+] eigenvectors fully recovered in {t4 -t3} seconds.")
    XT = matrix([v[0] for v in L1]) * C
    for i in range(n):
        if any(c==2 for c in XT[i]): XT[i] = -XT[i]
    if verbose: print(f"    [+] Number of recovered xi vectors {XT.nrows()}")
    if XT.nrows() < n:
        return False, L
    X =  XT.T
    # h = X * a
    a = matrix(Zmod(mod) ,X[:n]).inverse() * vector(Zmod(mod),h[:n])
    return True, a

def multivariate_attack(h, m, n, M, verbose=True):
    # C = optimized_multivariate_ol_attack_step1(h, m, n, M, verbose)
    if verbose: print("[*] Step 1, recovering X basis...")
    t0 = time.time()
    C = optimized_multivariate_ol_attack_step1_GF3(h, m, n, M, verbose)
    t1 = time.time()
    if verbose: print(f"    [+] Total Step 1 : {t1-t0} seconds")
    if verbose: print(f"[*] Step 2 multivariate equation solving...")
    a = _multivariate_attack(C, m, n, M, verbose)
    t2 = time.time()
    if verbose: print(f"    [+] Total Step 2 : {t2-t1} seconds")
    if verbose: print(f"[+] Total Attack {t2-t0} seconds")
    return a

In [27]:
n = 190

if n % 2==1:
    m = n * (n + 3) // 2 # n is odd
else:
    m = n * (n + 4) // 2 # n is even

Mbits = derive_mod_bits(n)
M, a, X, h  = gen_hssp(n, m, Mbits)

if n <= 32:
    C = multivariate_ol_attack_step1(h, m, n, M)
    XT_lattice = IntegerLattice(X.T)
    for row in C:
        assert (row in XT_lattice)
    print("ok with multivariate_ol_attack_step1")
    
    # C1 = _optimized_multivariate_ol_attack_step1(h, m, n, M)
    C1 = optimized_multivariate_ol_attack_step1(h, m, n, M)

    XT_lattice = IntegerLattice(X.T)
    for row in C1:
        assert (row in XT_lattice)
    print("ok with optimized_multivariate_ol_attack_step1")

In [4]:
find, recovered_a = multivariate_attack(h, m, n, M)
if find:
    print(f"[+] check {sorted(a) == sorted(recovered_a)}")

[*] Step 1, recovering X basis...
    [+] start to compute basis orthogonal to h
    [+] sub kernel LLL with dim ((380, 380)) costs 56.67947268486023 seconds
    [+] basis orthogonal to h computed in 345.33272767066956 seconds
    [*] start to LLL reduction with dim (380, 570)
    [+] LLL reduction done in 44.68091869354248
    [+] find 190 orthogonal vectors for 190 380-dim vectors
    [+] Total Step 1 : 413.3310739994049 seconds
[+] Step 2 multivariate equation solving...
    [+] ker.dimensions() = (190, 18335) , R matrix construction and kernel computation cost 287.37419986724854 seconds
    [+] systematic matrix and M_prime matrix constructed in 10.674431324005127 seconds.
    [+] eigenvectors fully recovered in 0.3937642574310303 seconds.
    [+] Number of recovered xi vectors 190
    [+] Total Step 2 421.53102445602417 seconds
[+] Total Attack 834.8620984554291 seconds
[+] check True


# Paper Implementation With Flatter 

I used `flatter` as a faster replacement for `LLL` in previous codes. Therefore, the implementation in this notebook is much fater than it in original paper.

For example, $n = 190$ , the running time recorded in paper is 46 minutes. However, using `flatter` instead of sage's built-in `LLL`, my implementation costs about only 14 minutes (without size-reduction optimization) and the paper implementation (with size-reduction optimization) with `flatter` costs about only 7 minutes.

In [29]:
# -*- mode: python;-*-

# This is an implementation of the Nguyen-Stern algorithm, and of our new multivariate attack

# To run the Nguyen-Stern algorithm, run the function NSattack(). 
# To run all the experiments with the Nguyen-Stern algorithm, run the function statNS().
# We provide the experimental results below.

# To run our algorithm, run the function multiAttack().
# To run all the experiments with our algorithm, run the function statMulti().
# We provide the experimental results below.

from time import time

# https://github.com/Neobeo/HackTM2023/blob/main/solve420.sage
def flatter(M):
    from subprocess import check_output
    from re import findall
    M = matrix(ZZ,M)
    # compile https://github.com/keeganryan/flatter and put it in $PATH
    z = '[[' + ']\n['.join(' '.join(map(str,row)) for row in M) + ']]'
    ret = check_output(["flatter"], input=z.encode())
    return matrix(M.nrows(), M.ncols(), map(int,findall(b'-?\\d+', ret)))

def genpseudoprime(eta,etamin=211):
    if eta<=(2*etamin):
        return random_prime(2**eta,False,2**(eta-1))
    else:
        return random_prime(2**etamin,False,2**(etamin-1))*genpseudoprime(eta-etamin)

def genParams(n=10,m=20,nx0=100):
    #print "Generation of x0",
    t=time()
    x0=genpseudoprime(nx0)
    #print time()-t

    # We generate the alpha_i's
    a=vector(ZZ,n)
    for i in range(n):
        a[i]=mod(ZZ.random_element(x0),x0)

    # The matrix X has m rows and must be of rank n
    while True:
        X=Matrix(ZZ,m,n)
        for i in range(m):
            for j in range(n):
                X[i,j]=ZZ.random_element(2)
        if X.rank()==n: break

    # We generate an instance of the HSSP: b=X*a
    c=vector(ZZ,[0 for i in range(m)])
    s=ZZ.random_element(x0)
    b=X*a
    for i in range(m):
        b[i]=mod(b[i],x0)

    return x0,a,X,b

# We generate the lattice of vectors orthogonal to b modulo x0
def orthoLattice(b,x0):
    m=b.length()
    M=Matrix(ZZ,m,m)

    for i in range(1,m):
        M[i,i]=1
    M[1:m,0]=-b[1:m]*inverse_mod(b[0],x0)
    M[0,0]=x0

    for i in range(1,m):
        M[i,0]=mod(M[i,0],x0)

    return M

def allones(v):
    if len([vj for vj in v if vj in [0,1]])==len(v):
        return v
    if len([vj for vj in v if vj in [0,-1]])==len(v):
        return -v
    return None

def recoverBinary(M5):
    lv=[allones(vi) for vi in M5 if allones(vi)]
    n=M5.nrows()
    for v in lv:
        for i in range(n):
            nv=allones(M5[i]-v)
            if nv and nv not in lv:
                lv.append(nv)
            nv=allones(M5[i]+v)
            if nv and nv not in lv:
                lv.append(nv)
    return Matrix(lv)

def allpmones(v):
    return len([vj for vj in v if vj in [-1,0,1]])==len(v)

# Computes the right kernel of M using LLL.
# We assume that m>=2*n. This is only to take K proportional to M.height()
# We follow the approach from https://hal.archives-ouvertes.fr/hal-01921335/document
def kernelLLL(M):
    n=M.nrows()
    m=M.ncols()
    # if m<2*n: return M.right_kernel().matrix()
    K=2**(m//2)*M.height()
  
    MB=Matrix(ZZ,m+n,m)
    MB[:n]=K*M
    MB[n:]=identity_matrix(m)
  
    # MB2=MB.T.LLL().T
    MB2=flatter(MB.T).T
    
    assert MB2[:n,:m-n]==0
    Ke=MB2[n:,:m-n].T

    return Ke
        
# This is the Nguyen-Stern attack, based on BKZ in the second step
def hssp_ns_attack(h, m, n, x0):
    b = h
    M=orthoLattice(b,x0)

    t=walltime()
    # M2=M.LLL()
    M2=flatter(M)
    print("LLL step1: %.1f" % walltime(t))

    MOrtho=M2[:m-n]

    print("  log(Height,2)=",int(log(MOrtho.height(),2)))

    t2=walltime()
    ke=kernelLLL(MOrtho)

    print("  Kernel: %.1f" % walltime(t2))
    print("  Total step1: %.1f" % walltime(t))

    if n>170: return

    beta=2
    tbk=walltime()
    while beta<n:
        if beta==2:
            M5=flatter(ke)
            # M5=ke.LLL()
        else:
            M5=M5.BKZ(block_size=beta)

        # we break when we only get vectors with {-1,0,1} components
        if len([True for v in M5 if allpmones(v)])==n: break

        if beta==2:
            beta=10
        else:
            beta+=10

    print("BKZ beta=%d: %.1f" % (beta,walltime(tbk)))
    t2=walltime()
    MB=recoverBinary(M5)
    print("  Recovery: %.1f" % walltime(t2))
    print("  Number of recovered vector=",MB.nrows())

    NS=MB.T
    invNSn=matrix(Integers(x0),NS[:n]).inverse()
    ra=invNSn*b[:n]
    print("  Total step2: %.1f" % walltime(tbk))
    print("  Total time: %.1f" % walltime(t))
    return ra
    
def test_hssp_ns_attack(n):
    m=int(max(2*n,16*log(n,2)))
    print("n=",n,"m=",m)

    iota=0.035
    nx0=int(2*iota*n**2+n*log(n,2))
    print("nx0=",nx0)

    x0,a,X,b=genParams(n,m,nx0)
    ra = hssp_ns_attack(b, m, n, x0)
    check = sorted(list(ra)) == sorted(list(a))
    print(f"{check = }")
    
# This is the Nguyen-Stern attack, based on BKZ in the second step
def NSattack(n=60):
    m=int(max(2*n,16*log(n,2)))
    print("n=",n,"m=",m)

    iota=0.035
    nx0=int(2*iota*n**2+n*log(n,2))
    print("nx0=",nx0)

    x0,a,X,b=genParams(n,m,nx0)
    ra = hssp_ns_attack(b, m, n, x0)
    check = sorted(list(ra)) == sorted(list(a))
    print(f"{check = }")

def statNS():
    for n in list(range(70,190,20)) + list(range(190,280,30)):
        NSattack(n)
        print()
    
test_hssp_ns_attack(64)

n= 64 m= 128
nx0= 670
LLL step1: 1.3
  log(Height,2)= 2
  Kernel: 1.5
  Total step1: 2.7
BKZ beta=10: 0.4
  Recovery: 0.3
  Number of recovered vector= 64
  Total step2: 0.7
  Total time: 3.4
check = True


In [30]:
def matNbits(M):
    return max([M[i,j].nbits() for i in range(M.nrows()) for j in range(M.ncols())])

# Matrix rounding to integers
def roundM(M):
    M2=Matrix(ZZ,M.nrows(),M.ncols())
    for i in range(M.nrows()):
        for j in range(M.ncols()):
            M2[i,j]=round(M[i,j])
    return M2

def orthoLatticeMod(b,n,x0):
    m=b.length()
    assert m>=3*n
    assert m % n==0
    M=Matrix(ZZ,m,3*n)
    M[:2*n,:2*n]=identity_matrix(2*n)
    for i in range(2,m//n):
        M[i*n:(i+1)*n,2*n:3*n]=identity_matrix(n)

    M[1:,0]=-b[1:]*inverse_mod(b[0],x0)
    M[0,0]=x0

    for i in range(1,m):
        M[i,0]=mod(M[i,0],x0)
    return M

def NZeroVectors(M):
    return sum([vi==0 and 1 or 0 for vi in M])

# This is our new multivariate attack        
def hssp_multiAttack(h, m, n, x0, k=4):

    print("n=",n,"m=",m,"k=",k)
    b = h

    M=orthoLatticeMod(b,n,x0)

    print("Step 1")
    t=walltime()

    # M[:n//k,:n//k]=M[:n//k,:n//k].LLL()
    M[:n//k,:n//k] = flatter(M[:n//k,:n//k])
    

    # M2 = M[:2*n,:2*n].LLL()
    M2 = flatter(M[:2*n,:2*n])
    
    tprecomp=walltime(t)
    print("  LLL:%.1f" % tprecomp)

    RF=RealField(matNbits(M))

    M4i=Matrix(RF,M[:n//k,:n//k]).inverse()
    M2i=Matrix(RDF,M2).inverse()

    ts1=walltime()
    while True:
        flag=True
        for i in range((m/n-2)*k):
            indf=2*n+n//k*(i+1)
            if i==(m//n-2)*k-1:
                indf=m

            mv=roundM(M[2*n+n//k*i:indf,:n//k]*M4i)
            if mv==0: 
                continue
            flag=False
            M[2*n+n//k*i:indf,:]-=mv*M[:n//k,:]
        if flag: break
    print("  Sred1:%.1f" % walltime(ts1))

    M[:2*n,:2*n]=M2

    ts2=walltime()
    while True:
        #print("  matNBits(M)=",matNbits(M[2*n:]))
        mv=roundM(M[2*n:,:2*n]*M2i)
        if mv==0: break
        M[2*n:,:]-=mv*M[:2*n,:]
    print("  Sred2:%.1f" % walltime(ts2))

    # Orthogonal of the orthogonal vectors
    # We compute modulo 3
    MO=Matrix(GF(3),n,m)

    tk=walltime()
    MO[:,:2*n]=kernelLLL(M[:n,:2*n])
    print("  Kernel LLL: %.1f" % walltime(tk))

    for i in range(2,m/n):
        MO[:,i*n:(i+1)*n]=-(M[i*n:(i+1)*n,:2*n]*MO[:,:2*n].T).T
    #print("Total kernel computation",walltime(tk))
    print("  Total Step 1: %.1f" % walltime(t))

    print("Step 2")
    t2=walltime()
    xt23=Matrix(GF(3),[(-x).list()+[x[i]*x[j]*((i==j) and 1 or 2) for i in range(n) for j in range(i,n)] for x in MO.T])
    ke3=xt23.right_kernel().matrix()
    print("  Kernel: %.1f" % walltime(t2))

    assert xt23.nrows()==m
    assert xt23.ncols()==n*(n+1)//2+n

    ke23=Matrix(GF(3),n,n*n)
    ind=n
    for i in range(n):
        for j in range(i,n):
            ke23[:,i*n+j]=ke3[:,ind]
            ke23[:,j*n+i]=ke3[:,ind]
            ind+=1

    tei=walltime()
    # We will compute the list of eigenvectors
    # We start with the full space.
    # We loop over the coordinates. This will split the eigenspaces.
    li=[Matrix(GF(3),identity_matrix(n))]    
    for j in range(n):       # We loop over the coordinates of the wi vectors.
        #print("j=",j)
        M=ke23[:,j*n:(j+1)*n]   # We select the submatrix corresponding to coordinate j
        li2=[]                 # We initialize the next list
        for v in li:
            if v.nrows()==1:     # We are done with this eigenvector 
                li2.append(v)
            else:     # eigenspace of dimension >1
                #print("eigenspace of dim:",v.nrows())
                A=v.solve_left(v*M)  # v*M=A*v. When we apply M on the right, this is equivalent to applying the matrix A.
                                      # The eigenvalues of matrix A correspond to the jth coordinates of the wi vectors in that
                                      # eigenspace
                for e,v2 in A.eigenspaces_left():    # We split the eigenspace according to the eigenvalues of A.
                    vv2=v2.matrix()
                    #print("  eigenspace of dim:",(vv2*v).nrows())
                    li2.append(vv2*v)                   # The new eigenspaces 

        li=li2

    #print("Eigenvectors computation",walltime(tei))

    NS=Matrix([v[0] for v in li])*MO
    for i in range(n):
        if any(c==2 for c in NS[i]): NS[i]=-NS[i]

    print("  Number of recovered vectors:",NS.nrows())

    NS=NS.T

    # b=X*a=NS*ra
    invNSn=matrix(Integers(x0),NS[:n]).inverse()
    ra=invNSn*b[:n]
    print("  Total step2: %.1f" % walltime(t2))
    print("  Total runtime: %.1f" % walltime(t))
    print()
    return ra
    
def test_multi_attack(n):
    if n % 2==1:
        m=n*(n+3)//2 # n is odd
    else:
        m=n*(n+4)//2 # n is even
    k=4
    iota=0.035
    nx0=int(2*iota*n**2+n*log(n,2))
    x0,a,X,b=genParams(n,m,nx0)
    ra = hssp_multiAttack(b, m, n, x0)
    check = sorted(list(ra)) == sorted(list(a))
    print(f"{check = }")

def multiAttack(n=16):
    if n % 2==1:
        m=n*(n+3)//2 # n is odd
    else:
        m=n*(n+4)//2 # n is even
    k=4

    print("n=",n,"m=",m,"k=",k)

    iota=0.035
    nx0=int(2*iota*n**2+n*log(n,2))
    print("nx0=",nx0)

    x0,a,X,b=genParams(n,m,nx0)
    ra = hssp_multiAttack(b, m, n, x0)
    check = sorted(list(ra)) == sorted(list(a))
    print(f"{check = }")

def statMulti():
    for n in list(range(70,210,20)) + [220,250]:
        multiAttack(n)
        print()

def _statMulti():
    for n in list(range(70,120,20)):
        multiAttack(n)
        print()
        
def _statNS():
    for n in list(range(70,120,20)):
        NSattack(n)
        print()

In [31]:
multiAttack(190)

n= 190 m= 18430 k= 4
nx0= 3965
n= 190 m= 18430 k= 4
Step 1
  LLL:69.6
  Sred1:65.2
  Sred2:24.1
  Kernel LLL: 45.9
  Total Step 1: 207.4
Step 2
  Kernel: 237.4
  Number of recovered vectors: 190
  Total step2: 251.0
  Total runtime: 458.4

check = True
